In [21]:
# Improt packages
import pandas as pd
import pyarrow
import json 

# Import functions from apps folder
from apps.collect_data import *

In [22]:
# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Women's_Prize_for_Fiction_winners"

books = get_wikidata(url)
books.head()


,Year,Author,Title,Result
0,2020,Maggie O'Farrell,Hamnet,Winner
1,2020,Angie Cruz,Dominicana,Finalist
2,2020,Bernardine Evaristo,"Girl, Woman, Other",Finalist
3,2020,Natalie Haynes,A Thousand Ships,Finalist
4,2020,Hilary Mantel,The Mirror and the Light,Finalist


In [23]:
books.to_parquet('assets/wpf.parquet')

## Info from book APIs

 Adding information from Google Book API and Open Library to the inital dataframe from the List of Womens Price for Fiction finalists and winners from the Wikipedapage 

In [24]:
# Collecting data from GBApi and OLApi
apidf = book_info_add(books)

In [25]:
# Merge dataframes 
book_df = pd.merge(apidf,
                     books,
                     on='Title', how='outer')

In [26]:
# storing the dataframe as parquet in the assets folder
book_df.to_parquet('assets/wpf_book_df.parquet')

In [25]:
# Checking that all books where merged by seing if there are any observations from the right side (no value in the Author(s) column)
book_df.loc[book_df['Author(s)'].isnull()==True]


,Title,Author(s),Publish_Date,Description,ISBN,Page_Count,Categories,Average_Rating,Rating_Count,Language,Year,Author,Result


In [26]:
book_df

,Title,Author(s),Publish_Date,Description,ISBN,Page_Count,Categories,Average_Rating,Rating_Count,Language,Year,Author,Result
0,Hamnet,Maggie O'Farrell,2023-04-15,'She's like no one I've ever met... She's like...,9781350416598,106.0,Drama,4.0,98.0,en,2020,Maggie O'Farrell,Winner
1,Dominicana,Angie Cruz,2019-09-03,A GOOD MORNING AMERICA BOOK CLUB PICK Shortlis...,9781250205926,381.0,Fiction,4.0,18.0,en,2020,Angie Cruz,Finalist
2,"Girl, Woman, Other",Bernardine Evaristo,2019-11-05,NATIONAL BESTSELLER WINNER OF THE BOOKER PRIZE...,9780802156990,464.0,Fiction,4.5,52.0,en,2020,Bernardine Evaristo,Finalist
3,A Thousand Ships,Natalie Haynes,2019-05-02,Shortlisted for the Women's Prize for Fiction ...,9781509836222,352.0,Fiction,4.0,15.0,en,2020,Natalie Haynes,Finalist
4,The Mirror and the Light,Hilary Mantel,2022-03-29,AN INTERNATIONAL BESTSELLER WINNER OF THE WALT...,1443413747,912.0,nan,4.5,42.0,en,2020,Hilary Mantel,Finalist
...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,The Leper's Companions,Julia Blackburn,2013-10-31,"To escape from her own sadness, a woman finds ...",9781448190904,224.0,Fiction,3.0,1.0,en,1999,Julia Blackburn,Shortlist
165,Visible Worlds,Marilyn Bowering,1999,Two brothers in 1930s Manitoba find themselves...,0006551130,308.0,Arctic regions,NaN,NaN,en,1999,Marilyn Bowering,Shortlist
166,The Short History of a Prince,Jane Hamilton,2010-09-01,Walter McCloud is a boy with dreams unlike mos...,9780307764072,368.0,Fiction,NaN,NaN,en,1999,Jane Hamilton,Shortlist
167,The Poisonwood Bible,Barbara Kingsolver,2008-09-04,**NOW INCLUDING THE FIRST CHAPTER OF DEMON COP...,9780571246212,633.0,Fiction,4.0,285.0,en,1999,Barbara Kingsolver,Shortlist


In [27]:
book_df.Categories.value_counts()

Categories
Fiction                           143
nan                                 5
Drama                               2
Domestic fiction                    2
Performing Arts                     1
Acquaintance rape                   1
Journalists                         1
Single women                        1
Actresses                           1
Alzheimer's disease                 1
African American men                1
Belfast (Northern Ireland)          1
Asians                              1
Book clubs (Discussion groups)      1
Biography & Autobiography           1
London (England)                    1
American fiction                    1
Blacks                              1
Corporations                        1
Law                                 1
Arctic regions                      1
Name: count, dtype: int64

In [28]:
# Collect topics for each book
book_dict = get_book_topics(book_df)

Error occurred for title: To be announced 14 June 2023, author: 


In [29]:

with open("assets/wpf_topics.json", "w") as outfile:
    json.dump(book_dict, outfile)

## My own Books from Goodreads export tool
Goodreads export using: https://www.goodreads.com/review/import

In [2]:
mybooks = pd.read_csv("assets/goodreads_library_export.csv")
mybooks = mybooks.rename(columns=lambda x: x.replace(' ', '_'))


In [8]:
to_read = mybooks.query("Exclusive_Shelf == 'to-read'")

In [10]:
# Collecting data from GBApi and OLApi
apimydf = book_info_add(mybooks)

In [17]:
# Merge dataframes 
my_book_df = pd.merge(mybooks,
                     apimydf,
                     on='Title', 
                     suffixes = ('_Goodreads', '_GoogleBooks'), 
                     how='left')

In [18]:
my_book_df.to_parquet('assets/my_book_df.parquet')

In [19]:
pd.set_option('display.max_columns',100)
my_book_df.head()

,Book_Id,Title,Author,Author_l-f,Additional_Authors,ISBN_Goodreads,ISBN13,My_Rating,Average_Rating_Goodreads,Publisher,Binding,Number_of_Pages,Year_Published,Original_Publication_Year,Date_Read,Date_Added,Bookshelves,Bookshelves_with_positions,Exclusive_Shelf,My_Review,Spoiler,Private_Notes,Read_Count,Owned_Copies,Author(s),Publish_Date,Description,ISBN_GoogleBooks,Page_Count,Categories,Average_Rating_GoogleBooks,Rating_Count,Language
0,6449422,"1Q84 (1Q84, #2)",Haruki Murakami,"Murakami, Haruki",NaN,"=""4103534230""","=""9784103534235""",5,4.10,Shinchosha/Tsai Fong Books,Hardcover,397.0,2009,2009.0,NaN,2023/04/28,NaN,NaN,read,NaN,NaN,NaN,1,0,"Haruki Murakami, Lica Hashimoto",NaN,"Nesse segundo volume, duas histórias em parale...",8579622050,376.0,Japanese fiction,NaN,NaN,pt-BR
1,6443834,"1Q84 Book 1 (1Q84, #1)",Haruki Murakami,"Murakami, Haruki",Murakami Haruki,"=""4103534222""","=""9784103534228""",5,4.02,Shinchosha/Tsai Fong Books,Hardcover,554.0,2009,2009.0,NaN,2023/04/28,NaN,NaN,read,NaN,NaN,NaN,1,0,Haruki Murakami,2013-01-22,NATIONAL BESTSELLER • The year is 1984 and the...,9780307476463,0.0,Fiction,3.5,167.0,en
2,61344814,Death of a Bookseller,Alice Slater,"Slater, Alice",NaN,"=""""","=""""",0,3.82,Hodder & Stoughton,Kindle Edition,384.0,2023,2023.0,NaN,2023/05/05,to-read,to-read (#128),to-read,NaN,NaN,NaN,0,0,Alice Slater,2023-04-27,WOULD YOU KILL FOR A GOOD STORY? THE MOST ANTI...,9781529385359,482.0,Fiction,NaN,NaN,en
3,25189315,Smoke Gets in Your Eyes & Other Lessons from t...,Caitlin Doughty,"Doughty, Caitlin",NaN,"=""0393351904""","=""9780393351903""",0,4.21,W. W. Norton & Company,Paperback,254.0,2015,2014.0,NaN,2023/05/05,to-read,to-read (#127),to-read,NaN,NaN,NaN,0,0,Caitlin Doughty,2015-04-16,THE NEW YORK TIMES BESTSELLER 'Unforgettable ....,9781782111047,228.0,Social Science,4.0,44.0,en
4,39897752,Lost in the Never Woods,Aiden Thomas,"Thomas, Aiden",NaN,"=""125031397X""","=""9781250313973""",0,3.72,Swoon Reads,Hardcover,384.0,2021,2021.0,NaN,2023/05/05,to-read,to-read (#126),to-read,NaN,NaN,NaN,0,0,Aiden Thomas,2022-08-16,When children start to go missing in the local...,9780349429984,338.0,Young Adult Fiction,4.0,6.0,en


In [4]:
# Collect topics for my own books from OLapi
my_topics = get_book_topics(mybooks)

In [33]:
# Writes the topics as 
with open("assets/my_topics.json", "w") as outfile:
    json.dump(my_topics, outfile)